### Instalação do pandas e do diretório de geração de arquivos pdf

In [ ]:
%Pip install pandas openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached numpy-2.3.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.3 MB 1.3 MB/s eta 0:00:09
    --------------------------------------- 0.2/11.3 MB 2.8 MB/s eta 0:00:05
   -- ------------------------------------- 0.6/11.3 MB 5.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.8/11.3 MB 5.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.2/11.3 MB 5.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/11.3 MB 5.4 MB/s eta 0:00:02
   ------ --------------------------------- 1.9/11.3 MB 6.3 MB/s eta 0:00:02
   -------- ------------------------------- 2.4/11.3


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install fpdf2

     ---------------------------------------- 0.0/72.7 kB ? eta -:--:--
     ---------------- --------------------- 30.7/72.7 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 72.7/72.7 kB 790.4 kB/s eta 0:00:00
  Using cached pillow-11.3.0-cp311-cp311-win_amd64.whl.metadata (9.2 kB)
     ---------------------------------------- 0.0/114.6 kB ? eta -:--:--
     -------------------------------------- 114.6/114.6 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/251.7 kB ? eta -:--:--
   ------------------------------ --------- 194.6/251.7 kB 5.9 MB/s eta 0:00:01
   ---------------------------------------- 251.7/251.7 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ------ --------------------------------- 0.4/2.3 MB 7.8 MB/s eta 0:00:01
   ------------- -------------------------- 0.8/2.3 MB 10.0 MB/s eta 0:00:01
   -------------------- ------------------- 1.2/2.3 MB 9.2 MB/s eta 0:00:01
   ----------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Base da geração dos arquivos em PDF

In [ ]:
import pandas as pd
from fpdf import FPDF, XPos, YPos
import os

# ==============================================================================
# 1. Configurações Iniciais e de Estilo (AJUSTE O LINK AQUI)
# ==============================================================================

# **AJUSTE CRUCIAL:** Substitua o GID (ID da Aba) para a aba 'Base_app'
# EX: 'https://docs.google.com/spreadsheets/d/1Rzo5vut2s4-tGnc2FY9UxrRBTXl7BD5xxHzinRvANuM/export?format=csv&gid=123456789'
URL_DADOS = 'https://docs.google.com/spreadsheets/d/1Rzo5vut2s4-tGnc2FY9UxrRBTXl7BD5xxHzinRvANuM/edit?gid=1215413035#gid=1215413035'

PASTA_SAIDA = 'boletins_performance'
LIMITE_TESTE = 3 

# Cores e Constantes (Design inspirado na Rede Decisão)
COR_PRINCIPAL = (220, 30, 60)         
COR_SECUNDARIA = (51, 153, 255)       
COR_FUNDO_SECAO = (240, 240, 240)     
COR_TEXTO_CLARO = (255, 255, 255)     
COR_TEXTO_ESCURO = (0, 0, 0)          
NOTA_CORTE_PERIGO = 6.0              

# ... Funções auxiliares (desenhar_cabecalho_aluno e desenhar_tabela_notas) são mantidas ...
# ... Função gerar_boletim_pdf é mantida ...

# ==============================================================================
# 4. Fluxo Principal de Execução (COM LEITURA CORRIGIDA)
# ==============================================================================

def main():
    print(f"Iniciando a Geração de Boletins (Limite de Teste: {LIMITE_TESTE})...")

    if not os.path.exists(PASTA_SAIDA):
        os.makedirs(PASTA_SAIDA)
        print(f"Pasta de saída criada: '{PASTA_SAIDA}'")

    # --- INÍCIO DA LEITURA ROBUSTA: PRIORIZANDO A VÍRGULA ---
    try:
        # Tenta com o delimitador VÍRGULA, o que o erro de tokenização sugere
        print(f"Lendo com VÍRGULA (sep=',', engine='python')...")
        df = pd.read_csv(URL_DADOS, sep=',', encoding='utf-8', engine='python') 
        
    except Exception as e:
        # Se falhar com VÍRGULA, tenta com PONTO E VÍRGULA uma última vez
        try:
            print(f"Tentativa de fallback: Lendo com PONTO E VÍRGULA (sep=';', engine='python')...")
            df = pd.read_csv(URL_DADOS, sep=';', encoding='utf-8', engine='python') 
        except Exception as e_retry:
             print(f"\nERRO FATAL ao carregar os dados: {e_retry}")
             print("\nINSTRUÇÃO: A leitura falhou. O problema é o delimitador. Verifique o GID e o compartilhamento.")
             return 

    print(f"Dados Carregados com sucesso! Total de {len(df)} registros.")

    # 4.1. LIMPEZA DOS NOMES DAS COLUNAS
    df.columns = df.columns.str.strip() 

    # 4.2. RENOMEAÇÃO CRUCIAL (Mapeia nomes reais para nomes usados no código)
    df.rename(columns={
        'aluno': 'Nome do Aluno',             
        'Disciplina': 'Disciplina',           
        'media_antes': 'Media_Trimestre_1',
           
    }, inplace=True)
    
    # DEBUG: Mostra os nomes finais para ajudar em caso de erro
    print(f"Colunas prontas para uso: {df.columns.tolist()}")
    
    # 4.3. AGRUPAMENTO DOS DADOS (Por 'Nome do Aluno')
    try:
        grupos_por_aluno = df.groupby('Nome do Aluno')
    except KeyError:
        print(f"\nERRO: Coluna 'aluno' não encontrada. Verifique o delimitador ou o nome exato!")
        return
        
    print(f"Total de alunos ÚNICOS identificados: {len(grupos_por_aluno)}")


    # 4.4. ITERAÇÃO POR GRUPO DE ALUNO E GERAÇÃO DO PDF
    contador = 0
    
    for nome_aluno, dados_aluno_df in grupos_por_aluno:
        
        gerar_boletim_pdf(nome_aluno, dados_aluno_df, PASTA_SAIDA)
        
        contador += 1
        if contador >= LIMITE_TESTE:
            print(f"\n--- Limite de {LIMITE_TESTE} PDFs atingido. Interrompendo a execução. ---")
            break 

    print("\nProcesso concluído! Verifique a pasta '{PASTA_SAIDA}' para os boletins de teste.")

if __name__ == "__main__":
    main()

Iniciando a Geração de Boletins (Limite de Teste: 3)...
Lendo com VÍRGULA (sep=',', engine='python')...
Tentativa de fallback: Lendo com PONTO E VÍRGULA (sep=';', engine='python')...

ERRO FATAL ao carregar os dados: ';' expected after '"'

INSTRUÇÃO: A leitura falhou. O problema é o delimitador. Verifique o GID e o compartilhamento.


### Regras do racional

**Conjunto de matérias** - se ele tiver 2/1 critérios vermelhos vamos entrar no detalhe com os planos de ação;